# 8. XML-Parsing

- Parsing von XML-Korpora mit etree
- Namensräume
- Entitäten 
- Validierung mit Dokumenttypdefinitionen (DTD’s)



##  8.1 XML-Parsing in Python
* verschiedene Parsing-Strategien:
  * SAX-Parsing (Callback-basiert)
  * DOM-Parsing (Abbildung der Baumstruktur im Programm)
  * Pull-Parsing (imperatives Parsing der XML-Struktur)
* verschiedene XML-Parser in Python-Standardlibrary:
  * `xml.sax` (SAX-Parser)
  * `xml.minidom` (DOM-Parser)
  * `xml.dom.pulldom` (Pull-Parser für DOM-Parser)
  * __`xml.etree`__ (pythonic XML-DOM-Parser zur Extraktion von Inhalten aus XML-Dateien)
* weitere XML-Library:
  * __`BeautifulSoup`__ (XML Parser, ebenfalls zur Extraktion von Inhalten aus XML-Dateien verwendet)


### XML-Generierung mit etree: Element(), SubElement()

- Erzeugen von XML (Knoten)  mit etree

In [12]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

root = ET.Element('root')
message = ET.SubElement(root, 'message')
message.text = "Hello, world!"

# Raw output: print(ET.tostring(root)) or ET.dump(root)

# Pretty printing: (reparse with minidom that supports pretty print)
dom = xml.dom.minidom.parseString(ET.tostring(root))
print(dom.toprettyxml())

<?xml version="1.0" ?>
<root>
	<message>Hello, world!</message>
</root>



### XML-Parsing mit etree: parse(), getroot(), find()

- Einlesen eines XML-Dokuments und und Parsen der Baumstruktur mit etree

In [13]:
import xml.etree.ElementTree as ET

tree = ET.parse('buchkatalog.xml')
root = tree.getroot()

print(root.tag)
# Iterieren über Knoten:
for c in root:
    print("+--> " + c.tag)
    title = c.find('titel')
    verlag = c.find('verlag')
    preis = c.find('preis')
    print("|    +--> " + title.tag + ": " + title.text)
    print("|    +--> " + verlag.tag + ": " + verlag.text)
    print("|    +--> " + preis.tag + ": " + preis.text)

buchkatalog
+--> buch
|    +--> titel: Faust I
|    +--> verlag: Reclam
|    +--> preis: 4,95
+--> buch
|    +--> titel: Maria Stuart: Trauerspiel in fünf Aufzügen
|    +--> verlag: Suhrkamp
|    +--> preis: 5,50
+--> buch
|    +--> titel: Buddenbrooks: Verfall einer Familie
|    +--> verlag: Fischer
|    +--> preis: 12,00
+--> buch
|    +--> titel: Treffpunkt im Unendlichen
|    +--> verlag: Rowohlt
|    +--> preis: 9,99


### XML-Parsing mit BeautifulSoup (bs4):

In [15]:
from bs4 import BeautifulSoup
import codecs

with codecs.open("buchkatalog.xml", "r", "utf-8") as file:
    soup = BeautifulSoup(file) 
    #The variable soup now contains a BeautifulSoup object that you can use to traverse the root element.

title = soup.find('titel')
print(title)

<titel>Faust I</titel>


In [16]:
for title in soup.find_all("titel"):
    print(title)

<titel>Faust I</titel>
<titel>Maria Stuart: Trauerspiel in fünf Aufzügen</titel>
<titel>Buddenbrooks: Verfall einer Familie</titel>
<titel>Treffpunkt im Unendlichen</titel>


## 8.2 XML-Attribute
* Elemente können _Attribute_ enthalten
* Attribute fügen Elementen weitere Informationen hinzu
* Attribute sind Schlüssel-Wertpaare: `key="value"`
* im Allgemeinen für Metadaten
* Unterschied oftmals nicht eindeutig:

Genre als Attribut:
```xml
...
<buch genre="tragödie">
 <titel>Faust I</titel>
 <autor>
  <vorname>Johann Wolfgang</vorname>
  <nachname>von Goethe</nachname>
 </autor>
 <preis>4,95</preis>
 <verlag>Reclam</verlag>
</buch>
...
```

Genre als Element (Auszeichnung):
```xml
...
<buch>
 <genre>tragödie</genre>
 <titel>Faust I</titel>
 <autor>
  <vorname>Johann Wolfgang</vorname>
  <nachname>von Goethe</nachname>
 </autor>
 <preis>4,95</preis>
 <verlag>Reclam</verlag>
</buch>
...
```

### Attribute mit etree: attrib-Dictionary

- Hinzufügen von Attribut mit etree:

In [17]:
import xml.etree.ElementTree as ET

tree = ET.parse('buchkatalog.xml')
root = tree.getroot()

id = 1
for c in root:
    c.attrib['id'] = str(id)
    id += 1
ET.dump(root)

<buchkatalog>

  <buch id="1">
    <titel>Faust I</titel>
    <autor>
      <vorname>Johann Wolfgang</vorname>
      <nachname>von Goethe</nachname>
    </autor>
    <preis>4,95</preis>
    <verlag>Reclam</verlag>
  </buch>

  <buch id="2">
    <titel>Maria Stuart: Trauerspiel in fünf Aufzügen</titel>
    <autor>
      <vorname>Friedrich</vorname>
      <nachname>Schiller</nachname>
    </autor>
    <preis>5,50</preis>
    <verlag>Suhrkamp</verlag>
  </buch>

  <buch id="3">
    <titel>Buddenbrooks: Verfall einer Familie</titel>
    <autor>
      <vorname>Thomas</vorname>
      <nachname>Mann</nachname>
    </autor>
    <preis>12,00</preis>
    <verlag>Fischer</verlag>
  </buch>

  <buch id="4">
    <titel>Treffpunkt im Unendlichen</titel>
    <autor>
      <vorname>Klaus</vorname>
      <nachname>Mann</nachname>
    </autor>
    <preis>9,99</preis>
    <verlag>Rowohlt</verlag>
  </buch>

</buchkatalog>


##  8.3 XML-Namensräume
* Namensräume zur Unterscheidung verschiedener Elemente mit gleichen Elementnamen
* Namensräume sind Präfixe von Elementnamen: `<prefix:name>`
* Es können auch Standardnamensräume ohne Präfixe vergeben werden

Namensräume mit Präfix:
```xml
<buch>
 <titel>Eine kurze Geschichte der Zeit</titel>
 <p:autor xmlns:p="www.dhvlab.gwi.uni-muenchen.de/xml/person">
  <p:vorname>Stephan</p:vorname>
  <p:nachname>Hawking</p:nachname>
  <p:titel>Prof. Dr.</p:titel>
 </p:autor>
 <preis>9,99</preis>
 <verlag>Rowohlt</verlag>
 <genre>wissenschaft</genre>
</buch>
```

Standardnamensräume:
```xml
<buch>
 <titel>Eine kurze Geschichte der Zeit</titel>
 <autor xmlns="www.dhvlab.gwi.uni-muenchen.de/xml/person">
  <vorname>Stephan</vorname>
  <nachname>Hawking</nachname>
  <titel>Prof. Dr.</titel>
 </autor>
 <preis>9,99</preis>
 <verlag>Rowohlt</verlag>
 <genre>wissenschaft</genre>
</buch>
```


In [18]:
import xml.etree.ElementTree as ET

# Namespace-Dictionary:
ns = {'p': 'www.dhvlab.gwi.uni-muenchen.de/xml/person'}

xml = '''
<buch>
 <titel>Eine kurze Geschichte der Zeit</titel>
 <p:autor xmlns:p="www.dhvlab.gwi.uni-muenchen.de/xml/person">
  <p:vorname>Stephan</p:vorname>
  <p:nachname>Hawking</p:nachname>
  <p:titel>Prof. Dr.</p:titel>
 </p:autor>
 <preis>9,99</preis>
 <verlag>Rowohlt</verlag>
 <genre>wissenschaft</genre>
</buch>
'''

root = ET.fromstring(xml)
print(root.tag + ": " + root.find('titel').text)
autor = root.find('p:autor', ns)
vorname = autor.find('p:vorname', ns)
nachname = autor.find('p:nachname', ns)
titel = autor.find('p:titel', ns)
print(vorname.tag + ": " + vorname.text)
print(nachname.tag + ": " + nachname.text)
print(titel.tag + ": " + titel.text)


buch: Eine kurze Geschichte der Zeit
{www.dhvlab.gwi.uni-muenchen.de/xml/person}vorname: Stephan
{www.dhvlab.gwi.uni-muenchen.de/xml/person}nachname: Hawking
{www.dhvlab.gwi.uni-muenchen.de/xml/person}titel: Prof. Dr.


In [19]:
import xml.etree.ElementTree as ET

# alternativ ohne Namespace-Dictionary:

xml = '''
<buch>
 <titel>Eine kurze Geschichte der Zeit</titel>
 <p:autor xmlns:p="www.dhvlab.gwi.uni-muenchen.de/xml/person">
  <p:vorname>Stephan</p:vorname>
  <p:nachname>Hawking</p:nachname>
  <p:titel>Prof. Dr.</p:titel>
 </p:autor>
 <preis>9,99</preis>
 <verlag>Rowohlt</verlag>
 <genre>wissenschaft</genre>
</buch>
'''

root = ET.fromstring(xml)
print(root.tag + ": " + root.find('titel').text)
autor = root.find('{www.dhvlab.gwi.uni-muenchen.de/xml/person}autor')
vorname = autor.find('{www.dhvlab.gwi.uni-muenchen.de/xml/person}vorname')
nachname = autor.find('{www.dhvlab.gwi.uni-muenchen.de/xml/person}nachname')
titel = autor.find('{www.dhvlab.gwi.uni-muenchen.de/xml/person}titel')
print(vorname.tag + ": " + vorname.text)
print(nachname.tag + ": " + nachname.text)
print(titel.tag + ": " + titel.text)


buch: Eine kurze Geschichte der Zeit
{www.dhvlab.gwi.uni-muenchen.de/xml/person}vorname: Stephan
{www.dhvlab.gwi.uni-muenchen.de/xml/person}nachname: Hawking
{www.dhvlab.gwi.uni-muenchen.de/xml/person}titel: Prof. Dr.


##  8.4 XML-Entitäten
* Entitäten in XML: `&entity-name;`
* Et-Zeichen (&): `&amp;`
* einfaches Anführungszeichen: `&apos;`
* doppeltes Anführungszeichen: `&quot;`
* größer als: `&gt;`
* kleiner als: `&lt;`


* Escapen durch XML-Entities von Zeichen, die Teil der XML-Syntax sind, ist notwendig, da sonst Syntaxfehler beim Parsing auftreten


* eigene Definitionen (Makros) möglich (mit Hilfe von DTDs)

In [1]:
import xml.etree.ElementTree as ET
doc = '''
<programm>
    if 1&lt;20:
        print(&apos;Eins ist kleiner als zwanzig!&apos;)
</programm>
'''
root = ET.fromstring(doc)
print(root.text)


    if 1<20:
        print('Eins ist kleiner als zwanzig!')



## 8.5 Dokumenttypdefinitionen (DTD)
* Schemasprache zur Festlegung von XML-Strukturen (z.B. TEI-Schema für Korpusdaten)
* Regeln aus welchen Elementen, Attributen und Entitäten ein
  XML-Dokument bestehen darf/muss
* sorgt für einheitliche Auszeichnung von Daten
* etwa für die Korpus-Dokumentenformat-Definitionen von TEI-XML verwendet (siehe Aufgabe 1)
* DTD's können in XML referenziert werden:
  * inline im Prolog des Dokuments
  * über private Datei: `<!DOCTYPE name SYSTEM "file.dtd">`
  * auf öffentliche DTD verweisen: `<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">`

### Elementdeklarationen

|Operator | Funktion|
|:---|:---|
|`#PCDATA` |  Textinhalt |
|`ANY` | Beliebiger Inhalt |
|`EMPTY` | Leeres Element |
|`Element` | Das Element muss genau einmal auftreten.|
|`Element*` | Das Element kann entweder gar nicht auftreten oder es kann beliebig oft auftreten.|
|`Element+` | Das Element muss mindestens einmal auftreten oder es kann beliebig oft auftreten.|
|`Element?` | Das Element kann entweder gar nicht auftreten oder es kann genau einmal auftreten.|
|`Element1,Element2,..` | Gibt eine festgelegte Sequenz von Elementen an.|
|`Element1`<code>&#124;</code>`Element2`<code>&#124;</code>`..` | Gibt eine Reihe von Alternativen an, aus der immer nur genau eine auftreten darf.|
|`(Element1 Op Element2..)` | Definiert eine Elementgruppe.|

### Attributdeklarationen

|Operator | Funktion|
|:---|:---|
|`#REQUIRED` | Attribut dar nicht ausgelassen werden. |
|`#IMPLIED` | Attribut kann ausgelassen werden. |
|`#FIXED "..."` | Attribut muss einen fixen Wert haben. |
|`"..."` | Standardwert, wenn Attribut ausgelassen wird. |
|`ID` | Ein eindeutiger Identifizierer.|
|`Character Data (CDATA)` | Zeichendaten, die nicht verarbeitet werden. |
|`(Wert1\`<code>&#124;</code>`Wert2`<code>&#124;</code>`...)` | Ein Wert aus einer Aufzählung. |

#### Beispiel:
```
<!ATTLIST Element1
 Attribut1 ID #REQUIRED
 Attribut2 CDATA #IMPLIED
>
```


#### Beispiel: programmsammlung.dtd
```xml
<!ELEMENT programmsammlung (programm+)>
<!ELEMENT programm ( name,sprache, code,autor)>
<!ELEMENT sprache (#PCDATA)>
<!ELEMENT name (#PCDATA)>
<!ELEMENT code (#PCDATA)>
<!ELEMENT autor (#PCDATA)>
```

#### programmsammlung.xml:
```xml
<programmsammlung>

	<programm>
		<name>simple_loop.py</name>
		<sprache>Python</sprache>
		<code>for p in persons:
			print(p.name + " " + p.age)
		</code>
		<autor>Jane Doe</autor>
	</programm>
	
	<programm>
		<name>tier.java</name>
		<sprache>Java</sprache>
		<code>public class Tier{
		private String art;
		private String name;
		
		public Tier(String art, String name){
			this.name = name;
			this.art = art; 
		}
		</code>
		<autor>John Doe</autor>
	</programm>
	
</programmsammlung>
```

### Validierung von XML-Dateien anhand von DTD-Schema
- die externe Python-Bibliothek [lxml](https://lxml.de) unterstützt
  DTD-Validation:


In [28]:
import sys
#!{sys.executable} -m pip install --upgrade pip lxml
from lxml import etree

with open('programmsammlung.dtd') as f:
    dtd = etree.DTD(f)
with open('programmsammlung.xml', 'rb') as f:
    root = etree.XML(f.read())

print(dtd.validate(root))

True


#### XML-Datei-interne DTD mit Definition von Makronamen (s.o. Entitäten):

In [29]:
doc = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE programm [
    <!ENTITY year "ⅯⅯⅩⅩⅠ">
    <!ENTITY longs "ſ">
    <!ENTITY cpr "Copyright by Erika &amp; Max Mu&longs;termann (&year;)">
]>
<programm>
    <name>simple_loop.py</name>
    <sprache>Python</sprache>
    <code>for p in persons:
            print(p.name + " " + p.age)
    </code>
    <copyright>&cpr;</copyright>
</programm>
'''
root = ET.fromstring(doc)
print("copyright: " + root.find('copyright').text)

copyright: Copyright by Erika & Max Muſtermann (ⅯⅯⅩⅩⅠ)


## Übungsaufgaben 8


### Aufgabe 1: xmlpos (Extraktion und Annotation von TEI-XML)
Schreiben Sie ein Python-Programm `xmlpos`, das aus einer Korpusdatei
den Dokumenteninhalt extrahiert, diesen dann satzweise POS-tagged
und eine XML-Datei folgender Form ausgibt:

```xml
<doc>
  <s id="s-0001">
    <w id="w-0001" pos="DET">Der</w>
	<w id="w-0002" pos="NOUN">Hase</w>
	...
  </s>
  <s id="s-0002">
  ...
</doc>
```

Anmerkungen:
* Verwenden Sie die [TEI](https://tei-c.org/)-kodierten Korpusdateien des [deutschen Textarchivs](https://www.deutschestextarchiv.de/), z.B:
  * Altman 1890: https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890
  * Brandes 1832: https://www.deutschestextarchiv.de/book/download_xml/brandes_naturlehre03_1832


> _XML-TEI (Text Encoding Initiative): weit verbreitetes XML-Dokumentenformat für die Kodierung von Textkorpora_    
  (Schema mit einheitlichen Elementnamen und Attributen für die editionswissenschaftliche und linguistische Annotation von Texten, u.a. über DTD definiert)

> _DTA (Deutsches Textarchiv): historisches deutsches Korpus_ (linguistisch annotiertes Volltextkorpus deutschsprachiger Texte aus der Zeit um 1600 bis 1900)


* Sie können die XML-Dateien auch mit Werkzeugen wie wget oder curl
  (man-Pages!)  herunterladen, oder auch Pythons `urllib` verwenden
* Sie können `nltk` zur Satzsegmentierung und zum POS-Tagging verwenden; alternativ
  dazu können Sie auch _stanza_ (oder _[spacy](https://spacy.io/)_) verwenden
* Stellen Sie sicher, dass der tatsächlichen Text aus den Dokumenten extrahiert wird;
  betrachten Sie hierzu folgendes (valides) XML-Dokument: `<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>` 

```xml
 <a>
	<b>
		1
		<c>
			2
			<d/>
			3
		</c>
	</b>
	4
	<lb/>
	5
</a> 
``` 



---
### Aufgabe 2: xmltcf (Verarbeitung von annotierten XML-Dateien im TCF-Format)

Im deutschen Textarchiv findet man auch annotierte Daten zu den
entsprechenden Dokumenten.  Laden Sie eine solche lemmatisierte und
annotierte XML-Datei im [TCF-Format](https://weblicht.sfs.uni-tuebingen.de/weblichtwiki/index.php/The_TCF_Format) (Text Corpus Format)
herunter (z.B. Altmann: https://www.deutschestextarchiv.de/book/download_fulltcf/16299)
und schreiben Sie ein Programm `xmltcf`, das eine zeilenweise die Sätze
ausgibt, wobei die Token entweder mit ihren Lemmata oder ihren Tags
annotiert sind:
```bash
$ ./xmltcf.py file.xml
Ich/ich gehe/gehen ...
$ ./xmltcf.py --tags file.xml
Der/DET Mann/NOUN ...
```
